In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import urllib.request
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm# Matplotlib and associated plotting modules
import matplotlib.colors as colors
from sklearn.cluster import KMeans# import k-means from clustering stage
import folium # map rendering library
import os
!pip install uszipcode
from uszipcode import SearchEngine
from datetime import date, timedelta

     |████████████████████████████████| 378 kB 6.9 MB/s 
     |████████████████████████████████| 77 kB 3.2 MB/s 


# Introduction

In this project we will explore San Francisco Neighborhoods and historical health violations in restaurants in each of them. For it we will access the City of San Francisco DataSF website and dataset available. 

We are looking to answer each of the following questions:
* 	What are the most common violations in this city?
* 	Can neighborhoods be clustered based on violation types to ensure effective deployment of marketing and specialists/packages?
* 	What are the most common food venue types in the neighborhoods?


# Step 1 : Getting Neighborhood information

First step will be to scrape the zipcodes and Neigborhoods from  http://www.healthysf.org/bdi/outcomes/zipmap.htm, this in order to obtain the zip codes we will use to find the latitude and longitude data points needed to plot the neighborhoods.

In [2]:
response = requests.get("http://www.healthysf.org/bdi/outcomes/zipmap.htm")
soup = BeautifulSoup(response.text, "lxml")
table = soup.find_all("table")
df = pd.read_html(str(table))
df = pd.DataFrame(df[4])
df.columns = df.iloc[0]
df = df.iloc[1:-1, :-1]
sf_neigh = df
sf_neigh.head()

,Zip Code,Neighborhood
1,94102,Hayes Valley/Tenderloin/North of Market
2,94103,South of Market
3,94107,Potrero Hill
4,94108,Chinatown
5,94109,Polk/Russian Hill (Nob Hill)


Now we will use the zip codes to find the needed latitude and longitude. We will use `uszipcode`, a database in Python that comes in handy for this purpose.

In [3]:
search = SearchEngine(simple_zipcode=True)
latitude=[]
longitude=[]

for index, row in sf_neigh.iterrows():
    zipcode= search.by_zipcode(row["Zip Code"]).to_dict()
    latitude.append(zipcode.get("lat"))
    longitude.append(zipcode.get("lng"))
    
sf_neigh["Latitude"] = latitude
sf_neigh["Longitude"] = longitude

sf_neigh.head()

Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!


,Zip Code,Neighborhood,Latitude,Longitude
1,94102,Hayes Valley/Tenderloin/North of Market,37.780,-122.420
2,94103,South of Market,37.780,-122.410
3,94107,Potrero Hill,37.770,-122.390
4,94108,Chinatown,37.791,-122.409
5,94109,Polk/Russian Hill (Nob Hill),37.790,-122.420


During early testing of mapping I noticed there is an issue with the location returned by `uszipcode` for the Outer Richmond neighborhood. I'll display the value returned by the python library. We will update this location to reflect correctly in the map

In [4]:
sf_neigh.loc[sf_neigh["Zip Code"]=='94121']

,Zip Code,Neighborhood,Latitude,Longitude
13,94121,Outer Richmond,37.8,-122.7


In [5]:
sf_neigh.loc[sf_neigh["Zip Code"]=='94121']=[['94121','Outer Richmond',37.781934,-122.4906]]
sf_neigh.loc[sf_neigh["Zip Code"]=='94121']

,Zip Code,Neighborhood,Latitude,Longitude
13,94121,Outer Richmond,37.781934,-122.4906


# Step 2 :  Create a map of San Francisco showing its Neighborhoods

We will use geopy to get the latitude and longitude for San Francisco

In [6]:
#I was using geopy originally but it has proven to be unstable. Hard coding to ensure stability and availability
latitude = 37.7749
longitude = -122.4194
print('The geograpical coordinates of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of San Francisco are 37.7749, -122.4194.


We will then use folium to plot the map and overlay the neighborhood markers

In [7]:
sf_df = sf_neigh
map_sf = folium.Map(location = [latitude, longitude], zoom_start=12, height=500, width=600)

# add markers to map
for lat, lng, neighborhood in zip(sf_df['Latitude'], sf_df['Longitude'], sf_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color = 'black',
        fill = True,
        fill_color = '#696969',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_sf)  
        
map_sf

# Step 3 : Data Exploration and Preparation for the violations dataset

We will use the Restaurant Scores dataset available at https://datasf.org/. The dataset contains records from October 2016 to November 2019. We will refer to data from 2019 only as it will be most relevant for our purposes.

Pull the dataset into a dataframe:

In [8]:
violations_df = pd.read_csv('https://data.sfgov.org/api/views/pyih-qa8i/rows.csv')
violations_df.shape

(53973, 23)

Drop all records older than January 1st, 2019. Drop columns we do not need. Drop rows with no zipcode or violation. Reset Index

In [9]:
violations_df['inspection_date'] = pd.to_datetime(violations_df['inspection_date'])
cutoff = '01/01/2019'
violations_df = violations_df.loc[(violations_df['inspection_date'] > cutoff)]
violations_df = violations_df.drop(['business_city','business_state','business_location','business_phone_number','inspection_id','violation_id','Neighborhoods (old)','Police Districts','Supervisor Districts','Fire Prevention Districts','Analysis Neighborhoods','business_latitude','business_longitude','inspection_score','inspection_type','Zip Codes'], axis=1)
violations_df.dropna(subset=['business_postal_code','violation_description'],inplace=True)
violations_df.reset_index(inplace=True, drop=True)

We will merge the neighborhood dataset and the violations one so we keep only data pertinent to the neighborhoods under analysis

In [10]:
sf_df = sf_df.merge(violations_df, left_on='Zip Code', right_on='business_postal_code')
sf_df.drop (['business_postal_code'], axis=1, inplace=True)
sf_df

,Zip Code,Neighborhood,Latitude,Longitude,business_id,business_name,business_address,inspection_date,violation_description,risk_category
0,94102,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,82225,Harry Harringtons Pub,460 Larkin St,2019-03-19,Improper or defective plumbing,Low Risk
1,94102,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,69705,Daniel's Cafe,154 Ellis St,2019-02-12,Moderate risk food holding temperature,Moderate Risk
2,94102,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,93072,Subway #68745,170 O'Farrell St #428,2019-01-03,Improper or defective plumbing,Low Risk
3,94102,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,69705,Daniel's Cafe,154 Ellis St,2019-02-12,Wiping cloths not clean or properly stored or ...,Low Risk
4,94102,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,72171,John's Grill,63 Ellis St,2019-01-22,Improper cooling methods,High Risk
...,...,...,...,...,...,...,...,...,...,...
11265,94134,Visitacion Valley/Sunnydale,37.72,-122.41,379,GIANT SCOOP,2708 SAN BRUNO Ave,2019-04-08,Unapproved or unmaintained equipment or utensils,Low Risk
11266,94134,Visitacion Valley/Sunnydale,37.72,-122.41,2622,WING HING RESTAURANT,2550 SAN BRUNO Ave,2019-01-24,Unclean or degraded floors walls or ceilings,Low Risk
11267,94134,Visitacion Valley/Sunnydale,37.72,-122.41,2785,G & L RESTAURANT,198 LELAND Ave,2019-05-02,Unapproved or unmaintained equipment or utensils,Low Risk
11268,94134,Visitacion Valley/Sunnydale,37.72,-122.41,194,BAYSIDE COFFEE SHOP,2011 BAYSHORE Blvd,2019-03-19,Inadequate and inaccessible handwashing facili...,Moderate Risk


Let us Explore different views of the data. Let's begin by understanding the breakdown of violation severity:

In [11]:
explore_df = sf_df.groupby('violation_description')['risk_category'].value_counts().unstack().fillna(0)
sum_column = explore_df["High Risk"] + explore_df["Moderate Risk"] + explore_df["Low Risk"]
explore_df["Total"] = sum_column
explore_df.sort_values(['Total'], ascending=False, inplace=True)
explore_df.head(10)

risk_category,High Risk,Low Risk,Moderate Risk,Total
violation_description,,,,
Inadequately cleaned or sanitized food contact surfaces,0.0,0.0,961.0,961.0
Unapproved or unmaintained equipment or utensils,0.0,896.0,0.0,896.0
Unclean or degraded floors walls or ceilings,0.0,847.0,0.0,847.0
Moderate risk food holding temperature,0.0,0.0,766.0,766.0
Inadequate and inaccessible handwashing facilities,0.0,0.0,760.0,760.0
Foods not protected from contamination,0.0,0.0,560.0,560.0
Wiping cloths not clean or properly stored or inadequate sanitizer,0.0,526.0,0.0,526.0
High risk food holding temperature,447.0,0.0,0.0,447.0
Moderate risk vermin infestation,0.0,0.0,443.0,443.0


In [12]:
explore_df.shape

(62, 4)

In [13]:
explore_df.sum()

risk_category
High Risk         1698.0
Low Risk          5102.0
Moderate Risk     4470.0
Total            11270.0
dtype: float64

In [14]:
explore_df.to_csv('explore.csv',index=True)

There are 62 violation types in our dataframe. A total of 11,270 records. 45% of violations in 2019 are considered low risk, 40% moderate and ~15% high risk. 

Most common violations are related to cleaning standards and food temperature holding. These findings will be relevant to determine solution packages we can offer to potential customers. As well as to understand areas of specialization we need to look for in potential hires joining the company.

# Step 4 : Analyze Neighborhoods and Violations

We will now shift our focus to each neighborhood and violatiosn in them. In order to do so we will group by violation description as shown below:

In [15]:
clustering_df= sf_df.groupby('Neighborhood')['violation_description'].value_counts().unstack().fillna(0)
clustering_df.head(10)

violation_description,Consumer advisory not provided for raw or undercooked foods,Contaminated or adulterated food,Discharge from employee nose mouth or eye,Employee eating or smoking,Food in poor condition,Food safety certificate or food handler card not available,Foods not protected from contamination,High risk food holding temperature,High risk vermin infestation,Improper cooking time or temperatures,...,Unauthorized or unsafe use of time as a public health control measure,Unclean hands or improper use of gloves,Unclean nonfood contact surfaces,Unclean or degraded floors walls or ceilings,Unclean or unsanitary food contact surfaces,Unclean unmaintained or improperly constructed toilet facilities,Unpermitted food facility,Unsanitary employee garments hair or nails,Wiping cloths not clean or properly stored or inadequate sanitizer,Worker safety hazards
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Bayview-Hunters Point,0.0,0.0,0.0,2.0,2.0,16.0,21.0,16.0,4.0,0.0,...,0.0,10.0,29.0,28.0,3.0,0.0,0.0,0.0,17.0,0.0
Castro/Noe Valley,1.0,0.0,0.0,2.0,1.0,11.0,21.0,13.0,2.0,0.0,...,0.0,1.0,2.0,28.0,6.0,0.0,1.0,0.0,29.0,0.0
Chinatown,1.0,1.0,0.0,2.0,0.0,15.0,24.0,29.0,22.0,0.0,...,1.0,8.0,21.0,35.0,14.0,4.0,0.0,0.0,26.0,0.0
Haight-Ashbury,0.0,0.0,0.0,2.0,0.0,13.0,12.0,10.0,8.0,0.0,...,0.0,2.0,3.0,29.0,14.0,1.0,0.0,0.0,10.0,0.0
Hayes Valley/Tenderloin/North of Market,2.0,6.0,0.0,3.0,3.0,32.0,27.0,42.0,18.0,0.0,...,0.0,5.0,39.0,56.0,29.0,2.0,0.0,1.0,40.0,0.0
Ingelside-Excelsior/Crocker-Amazon,0.0,1.0,0.0,7.0,1.0,15.0,26.0,34.0,21.0,0.0,...,2.0,23.0,20.0,45.0,19.0,1.0,0.0,0.0,16.0,0.0
Inner Mission/Bernal Heights,0.0,11.0,1.0,14.0,3.0,36.0,62.0,65.0,44.0,0.0,...,0.0,18.0,87.0,99.0,49.0,10.0,5.0,1.0,52.0,0.0
Inner Richmond,0.0,1.0,0.0,6.0,0.0,20.0,28.0,17.0,8.0,0.0,...,1.0,18.0,16.0,32.0,48.0,1.0,3.0,1.0,25.0,0.0
Lake Merced,0.0,0.0,0.0,0.0,0.0,6.0,23.0,5.0,1.0,0.0,...,0.0,3.0,4.0,22.0,7.0,0.0,0.0,0.0,27.0,0.0


We will now perform one hot encoding on the grouped data frame

In [16]:
#one hot encoding
sf_onehot=pd.get_dummies(sf_df[['violation_description']],prefix="", prefix_sep="")
#add neighborhood column back to the dataframe, move it ot the first column
sf_onehot['Neighborhood'] = sf_df['Neighborhood']
fixed_columns = [sf_onehot.columns[-1]] + list(sf_onehot.columns[:-1])
sf_onehot = sf_onehot[fixed_columns]

sf_onehot.head()

,Neighborhood,Consumer advisory not provided for raw or undercooked foods,Contaminated or adulterated food,Discharge from employee nose mouth or eye,Employee eating or smoking,Food in poor condition,Food safety certificate or food handler card not available,Foods not protected from contamination,High risk food holding temperature,High risk vermin infestation,...,Unauthorized or unsafe use of time as a public health control measure,Unclean hands or improper use of gloves,Unclean nonfood contact surfaces,Unclean or degraded floors walls or ceilings,Unclean or unsanitary food contact surfaces,Unclean unmaintained or improperly constructed toilet facilities,Unpermitted food facility,Unsanitary employee garments hair or nails,Wiping cloths not clean or properly stored or inadequate sanitizer,Worker safety hazards
0,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Take the one hot encoded dataframe, group by mean and reset the index:

In [17]:
sf_grouped = sf_onehot.groupby('Neighborhood').mean().reset_index()
sf_grouped.head()

,Neighborhood,Consumer advisory not provided for raw or undercooked foods,Contaminated or adulterated food,Discharge from employee nose mouth or eye,Employee eating or smoking,Food in poor condition,Food safety certificate or food handler card not available,Foods not protected from contamination,High risk food holding temperature,High risk vermin infestation,...,Unauthorized or unsafe use of time as a public health control measure,Unclean hands or improper use of gloves,Unclean nonfood contact surfaces,Unclean or degraded floors walls or ceilings,Unclean or unsanitary food contact surfaces,Unclean unmaintained or improperly constructed toilet facilities,Unpermitted food facility,Unsanitary employee garments hair or nails,Wiping cloths not clean or properly stored or inadequate sanitizer,Worker safety hazards
0,Bayview-Hunters Point,0.000000,0.000000,0.0,0.005291,0.005291,0.042328,0.055556,0.042328,0.010582,...,0.000000,0.026455,0.076720,0.074074,0.007937,0.000000,0.00000,0.000000,0.044974,0.0
1,Castro/Noe Valley,0.003460,0.000000,0.0,0.006920,0.003460,0.038062,0.072664,0.044983,0.006920,...,0.000000,0.003460,0.006920,0.096886,0.020761,0.000000,0.00346,0.000000,0.100346,0.0
2,Chinatown,0.001678,0.001678,0.0,0.003356,0.000000,0.025168,0.040268,0.048658,0.036913,...,0.001678,0.013423,0.035235,0.058725,0.023490,0.006711,0.00000,0.000000,0.043624,0.0
3,Haight-Ashbury,0.000000,0.000000,0.0,0.006849,0.000000,0.044521,0.041096,0.034247,0.027397,...,0.000000,0.006849,0.010274,0.099315,0.047945,0.003425,0.00000,0.000000,0.034247,0.0
4,Hayes Valley/Tenderloin/North of Market,0.002372,0.007117,0.0,0.003559,0.003559,0.037960,0.032028,0.049822,0.021352,...,0.000000,0.005931,0.046263,0.066429,0.034401,0.002372,0.00000,0.001186,0.047450,0.0


Function to return the most common violations in each neighborhood:

In [18]:
def return_most_common_violations(row, num_top):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top]

In [19]:
num_top = 10
indicators = ['st','nd','rd']

#create columns according to number of top violations
columns=['Neighborhood']
for ind in np.arange(num_top):
    try:
        #append indicators to top 3 venues
        columns.append('{}{} Most Common Violation'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Violation'.format(ind+1))
        
#set up the new dataframe
neighborhoods_violations_sorted = pd.DataFrame(columns=columns)
neighborhoods_violations_sorted['Neighborhood'] = sf_grouped['Neighborhood']

for ind  in np.arange(sf_grouped.shape[0]):
    neighborhoods_violations_sorted.iloc[ind,1:] = return_most_common_violations(sf_grouped.iloc[ind,:],num_top)
    
neighborhoods_violations_sorted.head()

,Neighborhood,1st Most Common Violation,2nd Most Common Violation,3rd Most Common Violation,4th Most Common Violation,5th Most Common Violation,6th Most Common Violation,7th Most Common Violation,8th Most Common Violation,9th Most Common Violation,10th Most Common Violation
0,Bayview-Hunters Point,Unapproved or unmaintained equipment or utensils,Inadequately cleaned or sanitized food contact...,Unclean nonfood contact surfaces,Unclean or degraded floors walls or ceilings,Foods not protected from contamination,Moderate risk food holding temperature,Inadequate and inaccessible handwashing facili...,Improper cooling methods,Wiping cloths not clean or properly stored or ...,Low risk vermin infestation
1,Castro/Noe Valley,Wiping cloths not clean or properly stored or ...,Unclean or degraded floors walls or ceilings,Inadequately cleaned or sanitized food contact...,Moderate risk food holding temperature,Inadequate and inaccessible handwashing facili...,Foods not protected from contamination,Unapproved or unmaintained equipment or utensils,Moderate risk vermin infestation,Low risk vermin infestation,High risk food holding temperature
2,Chinatown,Inadequately cleaned or sanitized food contact...,Inadequate and inaccessible handwashing facili...,Unapproved or unmaintained equipment or utensils,Moderate risk food holding temperature,Improper food storage,Unclean or degraded floors walls or ceilings,High risk food holding temperature,Wiping cloths not clean or properly stored or ...,Foods not protected from contamination,Low risk vermin infestation
3,Haight-Ashbury,Unapproved or unmaintained equipment or utensils,Unclean or degraded floors walls or ceilings,Moderate risk food holding temperature,Moderate risk vermin infestation,Inadequately cleaned or sanitized food contact...,Inadequate and inaccessible handwashing facili...,Unclean or unsanitary food contact surfaces,Food safety certificate or food handler card n...,Foods not protected from contamination,Improper storage of equipment utensils or linens
4,Hayes Valley/Tenderloin/North of Market,Inadequate and inaccessible handwashing facili...,Moderate risk food holding temperature,Inadequately cleaned or sanitized food contact...,Unapproved or unmaintained equipment or utensils,Unclean or degraded floors walls or ceilings,High risk food holding temperature,Wiping cloths not clean or properly stored or ...,Unclean nonfood contact surfaces,Improper food storage,Improper cooling methods


# Step 5 : Clustering neighborhoods according to violations

We will now apply the k means algorithm to the neighborhoods to be able to cluster them based on the most common violations:

In [20]:
from sklearn.cluster import KMeans
#set number of clusters
kclusters = 5
sf_grouped_clustering = sf_grouped.drop('Neighborhood',1)
#run k-means clustering
kmeans = KMeans (n_clusters = kclusters, random_state=0).fit(sf_grouped_clustering)
#check cluster labels
kmeans.labels_[0:10]

array([0, 1, 2, 4, 2, 2, 2, 4, 1, 2], dtype=int32)

Add the cluster labels to the original dataframe

In [21]:
neighborhoods_violations_sorted.insert(0,'Cluster Labels', kmeans.labels_)
sf_merged = sf_neigh
sf_merged = sf_merged.merge(neighborhoods_violations_sorted, on ='Neighborhood')
sf_merged.head()

,Zip Code,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Violation,2nd Most Common Violation,3rd Most Common Violation,4th Most Common Violation,5th Most Common Violation,6th Most Common Violation,7th Most Common Violation,8th Most Common Violation,9th Most Common Violation,10th Most Common Violation
0,94102,Hayes Valley/Tenderloin/North of Market,37.780,-122.420,2,Inadequate and inaccessible handwashing facili...,Moderate risk food holding temperature,Inadequately cleaned or sanitized food contact...,Unapproved or unmaintained equipment or utensils,Unclean or degraded floors walls or ceilings,High risk food holding temperature,Wiping cloths not clean or properly stored or ...,Unclean nonfood contact surfaces,Improper food storage,Improper cooling methods
1,94103,South of Market,37.780,-122.410,2,Unclean or degraded floors walls or ceilings,Inadequately cleaned or sanitized food contact...,Moderate risk food holding temperature,Unapproved or unmaintained equipment or utensils,Inadequate and inaccessible handwashing facili...,High risk food holding temperature,Foods not protected from contamination,Moderate risk vermin infestation,Wiping cloths not clean or properly stored or ...,Food safety certificate or food handler card n...
2,94107,Potrero Hill,37.770,-122.390,2,Moderate risk food holding temperature,Inadequately cleaned or sanitized food contact...,Unapproved or unmaintained equipment or utensils,Inadequate and inaccessible handwashing facili...,Unclean or degraded floors walls or ceilings,Foods not protected from contamination,Food safety certificate or food handler card n...,Wiping cloths not clean or properly stored or ...,High risk food holding temperature,Inadequate food safety knowledge or lack of ce...
3,94108,Chinatown,37.791,-122.409,2,Inadequately cleaned or sanitized food contact...,Inadequate and inaccessible handwashing facili...,Unapproved or unmaintained equipment or utensils,Moderate risk food holding temperature,Improper food storage,Unclean or degraded floors walls or ceilings,High risk food holding temperature,Wiping cloths not clean or properly stored or ...,Foods not protected from contamination,Low risk vermin infestation
4,94109,Polk/Russian Hill (Nob Hill),37.790,-122.420,2,Unapproved or unmaintained equipment or utensils,Inadequate and inaccessible handwashing facili...,Inadequately cleaned or sanitized food contact...,Moderate risk food holding temperature,Wiping cloths not clean or properly stored or ...,Unclean or degraded floors walls or ceilings,Foods not protected from contamination,Improper food storage,Unclean nonfood contact surfaces,High risk food holding temperature


Let's explore the defining characteristics of each cluster:

Cluster 1:

In [22]:
sf_merged.loc[sf_merged['Cluster Labels'] == 0, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Violation,2nd Most Common Violation,3rd Most Common Violation,4th Most Common Violation,5th Most Common Violation,6th Most Common Violation,7th Most Common Violation,8th Most Common Violation,9th Most Common Violation,10th Most Common Violation
15,Bayview-Hunters Point,Unapproved or unmaintained equipment or utensils,Inadequately cleaned or sanitized food contact...,Unclean nonfood contact surfaces,Unclean or degraded floors walls or ceilings,Foods not protected from contamination,Moderate risk food holding temperature,Inadequate and inaccessible handwashing facili...,Improper cooling methods,Wiping cloths not clean or properly stored or ...,Low risk vermin infestation
20,Visitacion Valley/Sunnydale,Unapproved or unmaintained equipment or utensils,High risk food holding temperature,Inadequately cleaned or sanitized food contact...,Inadequate and inaccessible handwashing facili...,Unclean nonfood contact surfaces,Moderate risk food holding temperature,Unclean or degraded floors walls or ceilings,Unclean hands or improper use of gloves,Improper thawing methods,Wiping cloths not clean or properly stored or ...


Cluster 2:

In [23]:
sf_merged.loc[sf_merged['Cluster Labels'] == 1, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Violation,2nd Most Common Violation,3rd Most Common Violation,4th Most Common Violation,5th Most Common Violation,6th Most Common Violation,7th Most Common Violation,8th Most Common Violation,9th Most Common Violation,10th Most Common Violation
7,Castro/Noe Valley,Wiping cloths not clean or properly stored or ...,Unclean or degraded floors walls or ceilings,Inadequately cleaned or sanitized food contact...,Moderate risk food holding temperature,Inadequate and inaccessible handwashing facili...,Foods not protected from contamination,Unapproved or unmaintained equipment or utensils,Moderate risk vermin infestation,Low risk vermin infestation,High risk food holding temperature
16,St. Francis Wood/Miraloma/West Portal,Foods not protected from contamination,Unclean or degraded floors walls or ceilings,Moderate risk food holding temperature,Inadequately cleaned or sanitized food contact...,Unapproved or unmaintained equipment or utensils,Wiping cloths not clean or properly stored or ...,Improper thawing methods,Unclean nonfood contact surfaces,Inadequate and inaccessible handwashing facili...,Low risk vermin infestation
17,Twin Peaks-Glen Park,Foods not protected from contamination,Wiping cloths not clean or properly stored or ...,Unclean or degraded floors walls or ceilings,Moderate risk food holding temperature,Unapproved or unmaintained equipment or utensils,Food safety certificate or food handler card n...,Low risk vermin infestation,Unclean or unsanitary food contact surfaces,Inadequately cleaned or sanitized food contact...,Improper thawing methods
18,Lake Merced,Wiping cloths not clean or properly stored or ...,Inadequate and inaccessible handwashing facili...,Foods not protected from contamination,Unclean or degraded floors walls or ceilings,Permit license or inspection report not posted,Inadequately cleaned or sanitized food contact...,Moderate risk food holding temperature,Improper thawing methods,Improper or defective plumbing,Unclean or unsanitary food contact surfaces


Cluster 3:

In [24]:
sf_merged.loc[sf_merged['Cluster Labels'] == 2, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Violation,2nd Most Common Violation,3rd Most Common Violation,4th Most Common Violation,5th Most Common Violation,6th Most Common Violation,7th Most Common Violation,8th Most Common Violation,9th Most Common Violation,10th Most Common Violation
0,Hayes Valley/Tenderloin/North of Market,Inadequate and inaccessible handwashing facili...,Moderate risk food holding temperature,Inadequately cleaned or sanitized food contact...,Unapproved or unmaintained equipment or utensils,Unclean or degraded floors walls or ceilings,High risk food holding temperature,Wiping cloths not clean or properly stored or ...,Unclean nonfood contact surfaces,Improper food storage,Improper cooling methods
1,South of Market,Unclean or degraded floors walls or ceilings,Inadequately cleaned or sanitized food contact...,Moderate risk food holding temperature,Unapproved or unmaintained equipment or utensils,Inadequate and inaccessible handwashing facili...,High risk food holding temperature,Foods not protected from contamination,Moderate risk vermin infestation,Wiping cloths not clean or properly stored or ...,Food safety certificate or food handler card n...
2,Potrero Hill,Moderate risk food holding temperature,Inadequately cleaned or sanitized food contact...,Unapproved or unmaintained equipment or utensils,Inadequate and inaccessible handwashing facili...,Unclean or degraded floors walls or ceilings,Foods not protected from contamination,Food safety certificate or food handler card n...,Wiping cloths not clean or properly stored or ...,High risk food holding temperature,Inadequate food safety knowledge or lack of ce...
3,Chinatown,Inadequately cleaned or sanitized food contact...,Inadequate and inaccessible handwashing facili...,Unapproved or unmaintained equipment or utensils,Moderate risk food holding temperature,Improper food storage,Unclean or degraded floors walls or ceilings,High risk food holding temperature,Wiping cloths not clean or properly stored or ...,Foods not protected from contamination,Low risk vermin infestation
4,Polk/Russian Hill (Nob Hill),Unapproved or unmaintained equipment or utensils,Inadequate and inaccessible handwashing facili...,Inadequately cleaned or sanitized food contact...,Moderate risk food holding temperature,Wiping cloths not clean or properly stored or ...,Unclean or degraded floors walls or ceilings,Foods not protected from contamination,Improper food storage,Unclean nonfood contact surfaces,High risk food holding temperature
5,Inner Mission/Bernal Heights,Inadequately cleaned or sanitized food contact...,Unclean or degraded floors walls or ceilings,Moderate risk food holding temperature,Unapproved or unmaintained equipment or utensils,Unclean nonfood contact surfaces,Moderate risk vermin infestation,Inadequate and inaccessible handwashing facili...,High risk food holding temperature,Foods not protected from contamination,Inadequate food safety knowledge or lack of ce...
6,Ingelside-Excelsior/Crocker-Amazon,Inadequately cleaned or sanitized food contact...,Unapproved or unmaintained equipment or utensils,Unclean or degraded floors walls or ceilings,Inadequate and inaccessible handwashing facili...,High risk food holding temperature,Moderate risk food holding temperature,Improper food storage,Foods not protected from contamination,Inadequate food safety knowledge or lack of ce...,Unclean hands or improper use of gloves
14,Marina,Food safety certificate or food handler card n...,Unapproved or unmaintained equipment or utensils,Unclean or degraded floors walls or ceilings,Inadequately cleaned or sanitized food contact...,Wiping cloths not clean or properly stored or ...,Inadequate and inaccessible handwashing facili...,High risk vermin infestation,Unclean nonfood contact surfaces,High risk food holding temperature,Foods not protected from contamination
19,North Beach/Chinatown,Inadequately cleaned or sanitized food contact...,Inadequate and inaccessible handwashing facili...,Unapproved or unma

Cluster 4:

In [25]:
sf_merged.loc[sf_merged['Cluster Labels'] == 3, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Violation,2nd Most Common Violation,3rd Most Common Violation,4th Most Common Violation,5th Most Common Violation,6th Most Common Violation,7th Most Common Violation,8th Most Common Violation,9th Most Common Violation,10th Most Common Violation
9,Parkside/Forest Hill,Unclean or degraded floors walls or ceilings,Foods not protected from contamination,Inadequate and inaccessible handwashing facili...,Inadequately cleaned or sanitized food contact...,Improper food storage,Moderate risk vermin infestation,High risk vermin infestation,Unclean nonfood contact surfaces,Unapproved or unmaintained equipment or utensils,Moderate risk food holding temperature
13,Sunset,Unclean or degraded floors walls or ceilings,Foods not protected from contamination,Inadequate and inaccessible handwashing facili...,Moderate risk food holding temperature,Wiping cloths not clean or properly stored or ...,Inadequately cleaned or sanitized food contact...,Improper food storage,High risk food holding temperature,High risk vermin infestation,Moderate risk vermin infestation


Cluster 5:

In [26]:
sf_merged.loc[sf_merged['Cluster Labels'] == 4, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Violation,2nd Most Common Violation,3rd Most Common Violation,4th Most Common Violation,5th Most Common Violation,6th Most Common Violation,7th Most Common Violation,8th Most Common Violation,9th Most Common Violation,10th Most Common Violation
8,Western Addition/Japantown,Unapproved or unmaintained equipment or utensils,Inadequately cleaned or sanitized food contact...,Moderate risk vermin infestation,Unclean or degraded floors walls or ceilings,Inadequate and inaccessible handwashing facili...,Wiping cloths not clean or properly stored or ...,Moderate risk food holding temperature,Low risk vermin infestation,Foods not protected from contamination,Improper food storage
10,Haight-Ashbury,Unapproved or unmaintained equipment or utensils,Unclean or degraded floors walls or ceilings,Moderate risk food holding temperature,Moderate risk vermin infestation,Inadequately cleaned or sanitized food contact...,Inadequate and inaccessible handwashing facili...,Unclean or unsanitary food contact surfaces,Food safety certificate or food handler card n...,Foods not protected from contamination,Improper storage of equipment utensils or linens
11,Inner Richmond,Unapproved or unmaintained equipment or utensils,Inadequately cleaned or sanitized food contact...,Unclean or unsanitary food contact surfaces,Inadequate and inaccessible handwashing facili...,Moderate risk food holding temperature,Moderate risk vermin infestation,Unclean or degraded floors walls or ceilings,Improper food storage,Foods not protected from contamination,Improper thawing methods
12,Outer Richmond,Inadequately cleaned or sanitized food contact...,Unapproved or unmaintained equipment or utensils,Inadequate and inaccessible handwashing facili...,Moderate risk food holding temperature,Improper thawing methods,Wiping cloths not clean or properly stored or ...,Improper food storage,Unclean or unsanitary food contact surfaces,Moderate risk vermin infestation,Low risk vermin infestation


Now, we will plot the clusters in the map

In [27]:
# create map
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 12, height=500, width=600)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sf_merged['Latitude'], sf_merged['Longitude'], sf_merged['Neighborhood'], sf_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 10,
        popup = label,
        tooltip=label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters


# Step 6 : Exploring the market with Foursquare

In order to explore the marketplace and find the most common food places in each neighbohood we will use the Foursquare API:

In [28]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("CLIENT_ID")
secret_value_1 = user_secrets.get_secret("CLIENT_SECRET")

CLIENT_ID = user_secrets.get_secret("CLIENT_ID") # your Foursquare ID
CLIENT_SECRET = user_secrets.get_secret("CLIENT_SECRET") # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

Function to pull all venues within a 500m radius. By passing the 'Food' Category ID, we're filtering only food places at the API level:

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [30]:
sf_venues = getNearbyVenues(names = sf_neigh['Neighborhood'],
                                   latitudes = sf_neigh['Latitude'],
                                   longitudes = sf_neigh['Longitude']
                                  )
                                  
sf_venues.shape

(690, 7)

Let's look at how many venues we have for each neighborhood:

In [31]:
sf_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bayview-Hunters Point,3,3,3,3,3,3
Castro/Noe Valley,31,31,31,31,31,31
Chinatown,41,41,41,41,41,41
Haight-Ashbury,13,13,13,13,13,13
Hayes Valley/Tenderloin/North of Market,57,57,57,57,57,57
Ingelside-Excelsior/Crocker-Amazon,42,42,42,42,42,42
Inner Mission/Bernal Heights,38,38,38,38,38,38
Inner Richmond,45,45,45,45,45,45
Lake Merced,16,16,16,16,16,16


In [32]:
print('There are {} uniques categories.'.format(len(sf_venues['Venue Category'].unique())))

There are 76 uniques categories.


We need to turn the venue category into dummy variables

In [33]:
# one hot encoding
sf_onehot = pd.get_dummies(sf_venues[['Venue Category']], prefix = "", prefix_sep = "")

# add neighborhood column back to dataframe
sf_onehot['Neighborhood'] = sf_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sf_onehot.columns[-1]] + list(sf_onehot.columns[:-1])
sf_onehot = sf_onehot[fixed_columns]

sf_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burmese Restaurant,...,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taco Place,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


We will now group neighborhoods taking the mean of the frequency of each venue category

In [34]:
sf_grouped = sf_onehot.groupby('Neighborhood').mean().reset_index()
sf_grouped.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burmese Restaurant,...,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taco Place,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Bayview-Hunters Point,0.0,0.000000,0.00000,0.0,0.666667,0.0,0.00000,0.000000,0.0,...,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,Castro/Noe Valley,0.0,0.032258,0.00000,0.0,0.000000,0.0,0.00000,0.064516,0.0,...,0.000000,0.0,0.00000,0.032258,0.032258,0.032258,0.096774,0.0,0.000000,0.000000
2,Chinatown,0.0,0.048780,0.02439,0.0,0.024390,0.0,0.02439,0.024390,0.0,...,0.024390,0.0,0.04878,0.073171,0.000000,0.000000,0.024390,0.0,0.024390,0.048780
3,Haight-Ashbury,0.0,0.000000,0.00000,0.0,0.153846,0.0,0.00000,0.000000,0.0,...,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.076923,0.0,0.000000,0.000000
4,Hayes Valley/Tenderloin/North of Market,0.0,0.052632,0.00000,0.0,0.035088,0.0,0.00000,0.035088,0.0,...,0.035088,0.0,0.00000,0.052632,0.000000,0.017544,0.070175,0.0,0.052632,0.087719


Function to return most common venues:

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

We will explore top 4 food venue types for each neighborhood:

In [36]:
num_top_venues = 4

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        # append 'st', 'nd', 'rd' to the top 3 venues
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = sf_grouped['Neighborhood']

for ind in np.arange(sf_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sf_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Bayview-Hunters Point,Bakery,Food,Vietnamese Restaurant,Fried Chicken Joint
1,Castro/Noe Valley,Deli / Bodega,Mediterranean Restaurant,Thai Restaurant,Pizza Place
2,Chinatown,Café,Chinese Restaurant,Sushi Restaurant,Italian Restaurant
3,Haight-Ashbury,Bakery,Café,Indian Restaurant,Burrito Place
4,Hayes Valley/Tenderloin/North of Market,Sandwich Place,Vietnamese Restaurant,Thai Restaurant,Deli / Bodega
5,Ingelside-Excelsior/Crocker-Amazon,Pizza Place,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant
6,Inner Mission/Bernal Heights,Mexican Restaurant,Deli / Bodega,Italian Restaurant,Café
7,Inner Richmond,Sushi Restaurant,Vietnamese Restaurant,Pizza Place,Japanese Restaurant
8,Lake Merced,Café,Food Truck,Sandwich Place,Pizza Place
9,Marina,Italian Restaurant,French Restaurant,American Restaurant,Thai Restaurant


In [37]:
neighborhoods_venues_sorted.to_csv('venues.csv',index=False)

# Conclusions
* Identifying most common violations in the city has been possible with the dataset available. Overall business strategy should be planned around the most common ocurrences. This data should be used to drive the marketing strategy along with areas of focus when hiring specialists to be deployed to the different neighborhoods
* Neighborhoods have been clustered based on most common violations and there seems to be a geographical alignment between them (worth digging deeper into possible causes). In any case these geographical alignment can help us tailor the marketing strategy in these neighborhoods in an effective manner
* Varied cuisine is found in San Francisco. Most common venues are Cafés, Italian, Chinese and Mexican Restaurants. Tailoring our offering to these common venues can help us increase market share 


# Additional Considerations
* COVID19 has impacted all industries and restaurants are not the exception, in fact, they have been particularly impacted. Special consideration to the new business landscape needs to be taken, understanding possible regulatory changes as well as financial struggles from restaurant owners which can decrease the appetite for specialized services
* Despite the fact filtering has been made to select ‘Food’ places in the Foursquare API. This API is not necessarily complete as not all restaurants will be registered and additionally other places of interest (e.g. School Cafeterias) will not be returned. Therefore, further market research can be done around other venue types. Out of scope for this exercise